In [1]:
import numpy as np
import nltk
from nltk.corpus import brown
import re
from gensim import models
nltk.download("brown")
nltk.download("universal_tagset")
nltk.download("punkt")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
sentences = brown.tagged_sents(tagset="universal")

In [3]:
def remove_comma(match_obj):
    if match_obj.group(2) is not None:
        return match_obj.group(1)+match_obj.group(3)
def preprocess_word(w):
    w = w.lower()
    w = re.sub("n't","",w)
    w = re.sub("ed$","ing",w)
    w = re.sub(r"([a-zA-Z])(,+)([a-zA-Z])", remove_comma, w)
    w = re.sub(r"([0-9]+)([,.]+)([0-9]+)", remove_comma, w)
    w = re.sub("[0-9]+","1",w)
    return w

In [4]:
words = []
for sentence in sentences:
    for word in sentence:
        words.append(preprocess_word(word[0]))
words = np.unique(np.sort(np.array(words)))

wordDict = {}
for i in range(len(words)):
    wordDict[words[i]] = i

In [5]:
words_path = 'words.npy'
w2v_path = 'w2v_embeddings.npy'

In [6]:
np.save(words_path,wordDict)

In [7]:
wordDict1 = np.load(words_path,allow_pickle=True)

In [8]:
wordDict1 == wordDict

True

In [9]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
def getVectorFromW2V(str):
    if str in wv:
        return wv[str]
    else:
        return None

w2v_embeddings = np.zeros((len(wordDict) + 1, 300))
for word,i in wordDict.items():
    w2v_vector = getVectorFromW2V(word)
    if w2v_vector is not None:
        w2v_embeddings[i] = w2v_vector
w2v_embeddings


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
np.save(w2v_path,w2v_embeddings)

In [19]:
w2v = np.load(w2v_path)

In [23]:
np.sum((w2v==w2v_embeddings).astype(np.int32)) == w2v.shape[0]*w2v.shape[1]

True